# State Shapefile Mapping
- To generate random coordinate using Nominatim specified by user, this can be city
- To check the city lies in which state, and thus it will become the region on interest

In [1]:
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import leafmap.foliumap as leafmap

In [5]:
# read file
df = gpd.read_file("../data/shapefile/polbnda_mys.shp")
df.head(n = len(df))

,f_code,coc,nam,laa,pop,ypc,adm_code,salb,soc,geometry
0,FA001,MYS,KELANTAN,GUA MUSANG,-99999999,0,UNK,UNK,MYS,"POLYGON ((101.58979 4.55280, 101.58880 4.55438..."
1,FA001,MYS,JOHOR,KOTA TINGGI,-99999999,0,UNK,UNK,MYS,"POLYGON ((103.72271 1.66005, 103.72056 1.66298..."
2,FA001,MYS,JOHOR,SEGAMAT,-99999999,0,UNK,UNK,MYS,"POLYGON ((102.70207 2.83330, 102.71049 2.83064..."
3,FA001,MYS,PAHANG,BERA,-99999999,0,UNK,UNK,MYS,"POLYGON ((102.70207 2.83330, 102.70186 2.83363..."
4,FA001,MYS,PAHANG,ROMPIN,-99999999,0,UNK,UNK,MYS,"POLYGON ((102.70289 3.31518, 102.70296 3.31524..."
...,...,...,...,...,...,...,...,...,...,...
231,FA001,MYS,SABAH,KUDAT,-99999999,0,UNK,UNK,MYS,"POLYGON ((117.33995 7.27316, 117.34082 7.27270..."
232,FA001,MYS,SABAH,SEMPORNA,-99999999,0,UNK,UNK,MYS,"POLYGON ((118.78781 4.58668, 118.78745 4.58640..."
233,FA001,MYS,SABAH,SEMPORNA,-99999999,0,UNK,UNK,MYS,"POLYGON ((118.76195 4.41550, 118.76195 4.41550..."
234,FA001,MYS,SABAH,SANDAKAN,-99999999,0,UNK,UNK,MYS,"POLYGON ((118.10109 5.90575, 118.10128 5.90518..."


- Using intersection idea to check whether a point lies in which district
- Then derive the shape and district and state name

In [ ]:
m = leafmap.Map(center=[df['Latitude'][0], df['Longitude'][0]], zoom=15, google_map="HYBRID")
m.add_points_from_xy(
    df, 
    layer_name=,
    x="Longitude", 
    y="Latitude",
    icon_names=['gear', 'map', 'leaf', 'globe'],
)
m